In [6]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn import preprocessing
from scipy import linalg
import matplotlib.pyplot as plt
from pykrige.uk import UniversalKriging

In [7]:
data = pd.read_csv('data/training_CNN_results_v3.csv')

In [8]:
clean_data = data[data["Name"] != 'Adafactor']

clean_data["Name"] = clean_data["Name"].replace(['SGD'],0.1)
clean_data["Name"] = clean_data["Name"].replace(['Adam'],0.3)
clean_data["Name"] = clean_data["Name"].replace(['RMSprop'],0.2)
clean_data["Name"] = clean_data["Name"].replace(['AdamW'],0.4)
clean_data["Name"] = clean_data["Name"].replace(['Adadelta'],0.5)
clean_data["Name"] = clean_data["Name"].replace(['Adagrad'],0.6)
clean_data["Name"] = clean_data["Name"].replace(['Adamax'],0.7)
clean_data["Name"] = clean_data["Name"].replace(['Nadam'],0.8)
clean_data["Name"] = clean_data["Name"].replace(['Ftrl'],0.9)

C:\Users\Lucas\AppData\Local\Temp\ipykernel_3512\2918968098.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data["Name"] = clean_data["Name"].replace(['SGD'],0.1)
C:\Users\Lucas\AppData\Local\Temp\ipykernel_3512\2918968098.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data["Name"] = clean_data["Name"].replace(['Adam'],0.3)
C:\Users\Lucas\AppData\Local\Temp\ipykernel_3512\2918968098.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [9]:
clean_data.to_csv('data/training_CNN_results_v3-1.csv', index=False)
clean_data.astype('float32')

,Name,Convoluted_Layers1,Convoluted_Filters1,Convoluted_Layers2,Convoluted_Filters2,Hidden_Layer1,Hidden_Layer2,Learning_Rate,Batch_Size,Loss
0,0.4,22.0,3.0,19.0,3.0,33.0,17.0,0.005487,63.0,0.039741
1,0.4,24.0,2.0,24.0,1.0,50.0,25.0,0.005466,127.0,0.051480
2,0.8,21.0,2.0,17.0,3.0,41.0,28.0,0.009989,99.0,0.043908
3,0.2,32.0,1.0,31.0,3.0,33.0,32.0,0.009013,78.0,0.074034
4,0.8,23.0,1.0,18.0,2.0,55.0,30.0,0.001348,106.0,0.068589
...,...,...,...,...,...,...,...,...,...,...
2138,0.7,22.0,1.0,29.0,1.0,57.0,25.0,0.002672,45.0,0.256532
2139,0.3,22.0,3.0,21.0,1.0,36.0,29.0,0.009747,67.0,0.062995
2140,0.6,29.0,3.0,19.0,3.0,40.0,32.0,0.000519,85.0,1.668306
2141,0.4,23.0,1.0,32.0,3.0,46.0,32.0,0.004172,43.0,0.049501


In [10]:
halfData = clean_data.sample(frac = 0.5)
otherHalfData = clean_data.drop(halfData.index)
clean_data = halfData


In [11]:
print(halfData)

      Name  Convoluted_Layers1  Convoluted_Filters1  Convoluted_Layers2  \
40     0.9                  31                    2                  31   
1239   0.8                  29                    1                  30   
479    0.9                  20                    1                  26   
2063   0.6                  31                    3                  17   
256    0.9                  23                    3                  28   
...    ...                 ...                  ...                 ...   
658    0.3                  18                    2                  17   
2131   0.7                  28                    3                  32   
60     0.2                  26                    2                  23   
545    0.5                  18                    1                  26   
2129   0.4                  25                    2                  29   

      Convoluted_Filters2  Hidden_Layer1  Hidden_Layer2  Learning_Rate  \
40                      2

In [12]:
loss_data = clean_data["Loss"]
loss_data.astype('float32')
loss_data = loss_data.to_numpy()

In [13]:
min_loc = np.argmin(loss_data)

In [14]:
np.set_printoptions(suppress=True)
all_data = otherHalfData.to_numpy()

In [15]:
data = halfData.drop('Loss', axis=1)
dataOnly = np.asarray(data)

In [16]:
print(data)

      Name  Convoluted_Layers1  Convoluted_Filters1  Convoluted_Layers2  \
40     0.9                  31                    2                  31   
1239   0.8                  29                    1                  30   
479    0.9                  20                    1                  26   
2063   0.6                  31                    3                  17   
256    0.9                  23                    3                  28   
...    ...                 ...                  ...                 ...   
658    0.3                  18                    2                  17   
2131   0.7                  28                    3                  32   
60     0.2                  26                    2                  23   
545    0.5                  18                    1                  26   
2129   0.4                  25                    2                  29   

      Convoluted_Filters2  Hidden_Layer1  Hidden_Layer2  Learning_Rate  \
40                      2

In [17]:
mn = np.append(dataOnly[min_loc],loss_data[min_loc])

In [18]:
kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)

In [19]:
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=True)

In [20]:
gp.fit(dataOnly, loss_data)

GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=1, nu=1.5),
                         n_restarts_optimizer=5, normalize_y=True)

In [21]:
from scipy.stats import norm

In [22]:
surrogate_values = []
y_mean_values = []
y_cov_values = []

In [23]:
for X in range (len(all_data)):
    y_mean, y_cov = gp.predict(all_data[X:X+1,:-1], return_cov=True)
    y_mean_values.append(y_mean[0])
    y_cov_values.append(y_cov[0][0])
    dist = norm(loc=0.0,scale=1.0)
    sigma_tilde = (y_cov[0][0])**(1/2)
    u = (loss_data[min_loc] - y_mean[0])/sigma_tilde
    ei = sigma_tilde*((u * dist.cdf(u))+dist.pdf(u))
    surrogate_values.append(-ei)

In [25]:
print(len(surrogate_values))
print(len(y_cov_values))
print(len(y_mean_values))

1013
1013
1013


In [30]:
print(surrogate_values)
#print(y_cov_values)
#print(y_mean_values)

[-0.15165511677158705, -0.1276667106977513, -0.12691964901163097, -0.12128630421904052, -0.12749693959429287, -0.13169216064997175, -0.12674655221299258, -0.1332928000236583, -0.12937599549270626, -0.10410850868573031, -0.12725064327341978, -0.12940415564444818, -0.13046250010947943, -0.12582052649101863, -0.12744178164096773, -0.12106601307315074, -0.08530321768048965, -0.1278195432762609, -0.13226228694017633, -0.1235495487712402, -0.09777688352971133, -0.12485672210065704, -0.1273965215817011, -0.12679652982673947, -0.1341330083932369, -0.10963059204467074, -0.12369922560668725, -0.1228576515993818, -0.12576187412136708, -0.1575223775508381, -0.1274064140767319, -0.13393614406299637, -0.1266439841410702, -0.1119228348272844, -0.1268561311025717, -0.13161800661113052, -0.12830790448351528, -0.12398923558464016, -0.129658962155955, -0.12744399481859703, -0.12739653271997484, -0.1119046746619795, -0.13217088365586374, -0.12639005420660523, -0.14899641021320081, -0.1361000876414098, -0.

In [29]:
surrogate_values

[-0.15165511677158705,
 -0.1276667106977513,
 -0.12691964901163097,
 -0.12128630421904052,
 -0.12749693959429287,
 -0.13169216064997175,
 -0.12674655221299258,
 -0.1332928000236583,
 -0.12937599549270626,
 -0.10410850868573031,
 -0.12725064327341978,
 -0.12940415564444818,
 -0.13046250010947943,
 -0.12582052649101863,
 -0.12744178164096773,
 -0.12106601307315074,
 -0.08530321768048965,
 -0.1278195432762609,
 -0.13226228694017633,
 -0.1235495487712402,
 -0.09777688352971133,
 -0.12485672210065704,
 -0.1273965215817011,
 -0.12679652982673947,
 -0.1341330083932369,
 -0.10963059204467074,
 -0.12369922560668725,
 -0.1228576515993818,
 -0.12576187412136708,
 -0.1575223775508381,
 -0.1274064140767319,
 -0.13393614406299637,
 -0.1266439841410702,
 -0.1119228348272844,
 -0.1268561311025717,
 -0.13161800661113052,
 -0.12830790448351528,
 -0.12398923558464016,
 -0.129658962155955,
 -0.12744399481859703,
 -0.12739653271997484,
 -0.1119046746619795,
 -0.13217088365586374,
 -0.12639005420660523,
 -0